In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from utils import session_config
os.environ['CUDA_VISIBLE_DEVICES'] = "1" # 사용 gpu 선택
session_config.setup_gpus(True, 0.5) # gpu 메모리 사용량 세팅

In [ ]:
%config Completer.use_jedi = False

In [ ]:
import tensorflow as tf

# load your model already built at phase 1.
model = tf.keras.models.load_model("models/cifar_classifier.h5")

# load your dataset with tf.data
cifar10 = tf.keras.datasets.cifar10
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y))
train_ds = train_ds.shuffle(buffer_size = 20000, reshuffle_each_iteration=False)
val_ds = train_ds.take(10000)
train_ds = train_ds.skip(10000)

# Define customizing function for your model training

In [ ]:
def custom_model(model):
    # your loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    loss_tracker = tf.keras.metrics.Mean(name="loss")
    acc = tf.keras.metrics.SparseCategoricalAccuracy(name="acc")

    class CustomModel(tf.keras.Model):
        def train_step(self, data):
            x, y = data
            with tf.GradientTape() as tape:
                y_pred = self(x, training=True)  # Forward pass
                # Compute our own loss
                loss = loss_fn(y, y_pred)
                l2_loss = [tf.nn.l2_loss(t) for t in model.trainable_variables]
                l2_loss = self.l2_reg * tf.math.reduce_sum(l2_loss)
                total_loss = loss + l2_loss
            # Compute gradients
            trainable_vars = self.trainable_variables
            gradients = tape.gradient(total_loss, trainable_vars)

            # Update weights
            self.optimizer.apply_gradients(zip(gradients, trainable_vars))

            # Compute our own metrics
            loss_tracker.update_state(loss)
            acc.update_state(y, y_pred)
            return {"loss": loss_tracker.result(), "acc": acc.result()}
        def test_step(self, data):
            # Unpack the data
            x, y = data
            # Compute predictions
            y_pred = self(x, training=False)
            # Updates the metrics tracking the loss
            loss = loss_fn(y, y_pred)

            # Update the metrics.
            loss_tracker.update_state(loss)
            acc.update_state(y, y_pred)

            # Return a dict mapping metric names to current value.
            # Note that it will include the loss (tracked in self.metrics).
            return {m.name: m.result() for m in self.metrics}
        @property
        def metrics(self):
            # We list our `Metric` objects here so that `reset_states()` can be
            # called automatically at the start of each epoch
            # or at the start of `evaluate()`.
            # If you don't implement this property, you have to call
            # `reset_states()` yourself at the time of your choosing.
            return [loss_tracker, acc]
    return CustomModel(model.input, model.output)

# Hyper-parameter tuning

주요한 하이퍼파라미터는 다음과 같다. 

- optimizer = ["ADAM_W, "RMSPROP"]
- learning rate = [1E-3, 1E-4, 1E-5]
- learning rate scheduler = [constant, piecewise decay, linear decay, cosine decay restart]
- batch_size = [32, 64, 128]
- weight_decay = [1E-5, 5E-5, 1E-4]

리스트에 포함된 파라미터값에 의해 총 2x3x4x3x3 = 216가지 경우의 수에 대해 실험할 수 있다. 각 실험에 대해 Early stopping 지점까지 학습하고 validation 정확도를 측정한다. 우리는 주요한 하이퍼파라미터를 Grid search 방식으로 사전에 결정한다. 이런 방식은 하이퍼파라미터 튜닝에서 더 낮은 차원의 파라미터 공간에 대한 탐색을 수행할 수 있다. 이는 하이퍼파라미터 최적화에 대한 약간의 타협을 통해 효율적인 파라미터 튜닝을 가능하게 한다.

In [ ]:
from utils.hparam_tune import Keras_Tuner

In [ ]:
log_dir = "/home/files/AI_project_templates/logs/tuner_test1"
ckpt_dir = "/home/files/AI_project_templates/ckpts/tuner_test1"
initial_model_path = "models/cifar_classifier.h5"

tuner = Keras_Tuner(log_dir, ckpt_dir, initial_model_path, custom_model, train_ds, val_ds, metric="accuracy")

In [ ]:
tuner.tuning()

In [ ]:

# model = tf.keras.models.load_model("ckpts/.../trained_model")
# test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y))
# test_ds = self.test_ds.batch(32).prefetch(tf.data.experimental.AUTOTUNE)
# model.evaluate(test_ds)